In [1]:
import os 
import openai 
from dotenv import load_dotenv 

load_dotenv() 

openai.api_key = os.getenv("OPENAI_API_KEY")  
model_name = "gpt-3.5-turbo"

In [3]:
from typing import List 
from pydantic import BaseModel, Field

In [4]:
class User: 
    def __init__(self, name: str, age: int, email: str): 
        self.name = name 
        self.age = age 
        self.email = email 

In [5]:
foo = User(name="Joe", age=32, email="joe@gmail.com")

In [6]:
foo.name

'Joe'

In [8]:
class pUser(BaseModel): 
    name: str 
    age: int 
    email: str

In [9]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [10]:
foo_p.name

'Jane'

In [12]:
# This will fail 
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

ValidationError: 1 validation error for pUser
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='bar', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing

In [13]:
class Class(BaseModel): 
    students: List[pUser] 

In [14]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [15]:
obj

Class(students=[pUser(name='Jane', age=32, email='jane@gmail.com')])

In [21]:
# Pydantic to OpenAI function definition 
class WeatherSearch(BaseModel): 
    # Doc string turns into description
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str = Field(description="airport code to get the weather for") 

In [17]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function 

In [19]:
weather_function = convert_pydantic_to_openai_function(WeatherSearch)

In [20]:
weather_function

{'name': 'WeatherSearch',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get the weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [22]:
# This is expected to get an error 
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")  
convert_pydantic_to_openai_function(WeatherSearch1)


{'name': 'WeatherSearch1',
 'description': '',
 'parameters': {'properties': {'airport_code': {'description': 'airport code to get weather for',
    'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [23]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [24]:
convert_pydantic_to_openai_function(WeatherSearch2)

{'name': 'WeatherSearch2',
 'description': 'Call this with an airport code to get the weather at that airport',
 'parameters': {'properties': {'airport_code': {'type': 'string'}},
  'required': ['airport_code'],
  'type': 'object'}}

In [25]:
from langchain_openai import ChatOpenAI 

In [27]:
model = ChatOpenAI(model=model_name) 
model.invoke("What is the weather in sf today?", functions=[weather_function])

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 71, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-b9299255-ad02-40b4-8b11-e74c82ce4d31-0', usage_metadata={'input_tokens': 71, 'output_tokens': 17, 'total_tokens': 88})

In [28]:
model_with_function = model.bind(functions=[weather_function]) 
model_with_function.invoke("What is the weather in sf today?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 71, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-3295301f-7676-49c4-b11d-c3f4dc58f47a-0', usage_metadata={'input_tokens': 71, 'output_tokens': 17, 'total_tokens': 88})

In [30]:
model_with_forced_function = model.bind(functions=[weather_function], function_call={"name":"WeatherSearch"}) 
model_with_forced_function.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 80, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-09a1ca9b-7392-4bb0-8f7c-7ae764530ba8-0', usage_metadata={'input_tokens': 80, 'output_tokens': 7, 'total_tokens': 87})

In [31]:
model_with_forced_function.invoke("hi!")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 75, 'total_tokens': 82}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-09481e1d-8be5-4869-affe-8aa297e5e93a-0', usage_metadata={'input_tokens': 75, 'output_tokens': 7, 'total_tokens': 82})

In [32]:
# Using in a chain 
from langchain.prompts import ChatPromptTemplate 
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"), 
    ("user", "{input}")
])

In [34]:
chain = prompt | model_with_function 
chain.invoke({"input": "What is the weather in sf?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 76, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-1e7761c2-7d49-4c73-abf0-d4b2de1dc76b-0', usage_metadata={'input_tokens': 76, 'output_tokens': 17, 'total_tokens': 93})

In [35]:
# Multiple functions 
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [36]:
functions = [
    convert_pydantic_to_openai_function(WeatherSearch),
    convert_pydantic_to_openai_function(ArtistSearch),
]

In [37]:
model_with_functions = model.bind(functions=functions)

In [38]:
model_with_functions.invoke("what is the weather in sf?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'WeatherSearch'}}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 117, 'total_tokens': 134}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-4afa95f8-9c66-40e4-9a7d-a53be92b1f76-0', usage_metadata={'input_tokens': 117, 'output_tokens': 17, 'total_tokens': 134})

In [39]:
model_with_functions.invoke("what are three songs by taylor swift?")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"artist_name":"Taylor Swift","n":3}', 'name': 'ArtistSearch'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 119, 'total_tokens': 140}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-d2f8483e-4339-4627-9f98-79e781e075d4-0', usage_metadata={'input_tokens': 119, 'output_tokens': 21, 'total_tokens': 140})

In [40]:
model_with_functions.invoke("hi!")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 112, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dc8731f6-833b-42a9-97a8-186b52d03d03-0', usage_metadata={'input_tokens': 112, 'output_tokens': 10, 'total_tokens': 122})